In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm

In [ ]:
class BeeDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
queen_train = np.load('stft_queen_train.npy')
nonqueen_train = np.load('stft_nonqueen_train.npy')

queen_val = np.load('stft_queen_val.npy')
nonqueen_val = np.load('stft_nonqueen_val.npy')

queen_test = np.load('stft_queen_test.npy')
nonqueen_test = np.load('stft_nonqueen_test.npy')

In [ ]:
queen_train = np.load('mfcc_queen_train.npy')
nonqueen_train = np.load('mfcc_NonQueen_train.npy')

queen_val = np.load('mfcc_queen_val.npy')
nonqueen_val = np.load('mfcc_NonQueen_val.npy')

queen_test = np.load('mfcc_queen_test.npy')
nonqueen_test = np.load('mfcc_nonqueen_test.npy')

In [ ]:
x_train = np.vstack((queen_train, nonqueen_train))
ones = np.ones(7000)
zeros = np.zeros(7000)
y_train = np.concatenate((ones, zeros))

In [ ]:
x_val = np.vstack((queen_val, nonqueen_val))
ones = np.ones(1000)
zeros = np.zeros(1000)
y_val = np.concatenate((ones, zeros))

In [ ]:
x_test = np.vstack((queen_test, nonqueen_test))
ones = np.ones(2000)
zeros = np.zeros(2000)
y_test = np.concatenate((ones, zeros))

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
train_dataset = BeeDataset(x_train, y_train, transform=transform)
val_dataset = BeeDataset(x_val, y_val, transform=transform)
test_dataset = BeeDataset(x_test, y_test, transform=transform)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
seed = 42
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
model = models.regnet_y_400mf(weights='IMAGENET1K_V1')

model.stem[0] = nn.Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
model = models.regnet_y_400mf(weights='IMAGENET1K_V1')

model.stem[0] = nn.Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1), bias=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

state_dict = torch.load('/kaggle/working/RegNet_MFCC_ 97_78.pth')

model.load_state_dict(state_dict)


if isinstance(model.fc, nn.Linear) and model.fc.out_features != 1:
    model.fc = nn.Linear(model.fc.in_features, 1)
    
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Training and validation loop
num_epochs = 200
train_accs = []
val_accs = []
train_losses = []
val_losses = []
best_val_accuracy = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_corrects = 0

    for train_images, train_labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        train_images = train_images.to(device)
        # train_labels = train_labels.long().to(device)
        
        train_labels = train_labels.float().unsqueeze(1)
        train_labels = train_labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(train_images)
        # _, preds = torch.max(outputs, 1)
        # loss = criterion(outputs, train_labels)
        
        preds = torch.sigmoid(outputs).round()
        loss = criterion(outputs, train_labels)
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * train_images.size(0)
        train_corrects += torch.sum(preds == train_labels.data)

    train_loss /= len(train_dataloader.dataset)
    train_acc = train_corrects.double() / len(train_dataloader.dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation"):
            val_images = val_images.to(device)
            # val_labels = val_labels.long().to(device)
            
            val_labels = val_labels.float().unsqueeze(1)
            val_labels = val_labels.to(device)
            outputs = model(val_images)
            # _, preds = torch.max(val_outputs, 1)
            # loss = criterion(val_outputs, val_labels)
            
            preds = torch.sigmoid(outputs).round()
            loss = criterion(outputs, val_labels)
            
            val_loss += loss.item() * val_images.size(0)
            val_corrects += torch.sum(preds == val_labels.data)

    val_loss /= len(val_dataloader.dataset)
    val_acc = val_corrects.double() / len(val_dataloader.dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

   
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

In [ ]:
# Load the best model
best_model = model
best_model.load_state_dict(torch.load('best_model.pth'))

# Predict on the test set
best_model.eval()
test_predictions = []
true_labels = []
with torch.no_grad():
    for test_images, test_labels in tqdm(test_dataloader, desc="Test Set"):
        test_images = test_images.to(device)
        #test_labels = test_labels.long().to(device)
            
        test_labels = test_labels.float().unsqueeze(1)
        test_labels = test_labels.to(device)
        
        outputs = best_model(test_images)
        # _, preds = torch.max(test_outputs, 1)
        preds = torch.sigmoid(outputs).round()
        
        test_predictions.extend(preds.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

# Evaluate the model on the test set
num_correct = sum([1 for i in range(len(test_predictions)) if test_predictions[i] == true_labels[i]])
test_accuracy = num_correct / len(test_predictions)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')